In [1]:
from trueskill import Rating, rate
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [2]:
def print_commanders(commanders):
    ratings = [rating.mu for rating in commanders.values()]
    rankings = sorted(range(len(ratings)), key=lambda i: ratings[i], reverse=True)
    for rank in rankings:
        print(f"{rank + 1}. {list(commanders.keys())[rank]} - {list(commanders.values())[rank]}")

In [ ]:
matches = [
    # 2025-06-29
    [
        "Julian", # Julian
        "Simon", # Simon
        "Sebastian" # Sebastian
    ],
    [
        "Simon", # Simon
        "Sebastian", # Sebastian
        "Julian" # Julian
    ],
    [
        "Sebastian", # Sebastian
        "Simon", # Simon
        "Julian" # Julian
    ],
    [
        "Simon", # Simon
        "Sebastian", # Sebastian
        "Julian" # Julian
    ],
    # 2025-07-02
    [
        "Julian", # Julian
        "Sebastian", # Sebastian
        "Simon" # Simon
    ],
    [
        "Simon", # Simon
        "Sebastian", # Sebastian
        "Julian" # Julian
    ]
]

In [4]:
commanders = list(set([commander for match in matches for commander in match]))
commanders = {commander: Rating() for commander in commanders}

In [5]:
print_commanders(commanders)

1. Sebastian - trueskill.Rating(mu=25.000, sigma=8.333)
2. Simon - trueskill.Rating(mu=25.000, sigma=8.333)
3. Julian - trueskill.Rating(mu=25.000, sigma=8.333)


In [6]:
vis_df = pd.DataFrame(columns=["Match", "Commander", "TrueSkill Rating", "TrueSkill StDev"])

for i, match in enumerate(matches):
    print("\nMatch", i + 1,":")
    
    print("Ratings before match:")
    print_commanders({commander: commanders[commander] for commander in match})

    print("\nWinner:", match[0])
    print("Losers:", end=" ")
    for losers in match[1:(len(match)-1)]:
        print(losers, end="; ")
    print(match[-1])
    
    teams = [{commander: commanders[commander]} for commander in match]
    ranks = [0] + [1] * (len(match) - 1)
    result = rate(teams, ranks=ranks)
    
    for team in result:
        for commander in team:
            commanders[commander] = team[commander]

    print("\nRatings after match:")
    print_commanders({commander: commanders[commander] for commander in match})

    current_match = pd.DataFrame({
        "Match": i + 1,
        "Commander": match,
        "TrueSkill Rating": [commanders[commander].mu for commander in match],
        "TrueSkill StDev": [commanders[commander].sigma for commander in match]
    })

    previous_matches = vis_df[(vis_df["Match"] == i) & (~vis_df["Commander"].isin(match))]
    previous_matches["Match"] = i + 1

    vis_df = pd.concat([vis_df, current_match, previous_matches], ignore_index=True)


Match 1 :
Ratings before match:
1. Julian - trueskill.Rating(mu=25.000, sigma=8.333)
2. Simon - trueskill.Rating(mu=25.000, sigma=8.333)
3. Sebastian - trueskill.Rating(mu=25.000, sigma=8.333)

Winner: Julian
Losers: Simon; Sebastian

Ratings after match:
1. Julian - trueskill.Rating(mu=30.109, sigma=6.735)
3. Sebastian - trueskill.Rating(mu=22.448, sigma=5.974)
2. Simon - trueskill.Rating(mu=22.443, sigma=5.972)

Match 2 :
Ratings before match:
3. Julian - trueskill.Rating(mu=30.109, sigma=6.735)
2. Sebastian - trueskill.Rating(mu=22.448, sigma=5.974)
1. Simon - trueskill.Rating(mu=22.443, sigma=5.972)

Winner: Simon
Losers: Sebastian; Julian

Ratings after match:
1. Simon - trueskill.Rating(mu=26.856, sigma=4.937)
3. Julian - trueskill.Rating(mu=24.544, sigma=4.878)
2. Sebastian - trueskill.Rating(mu=22.410, sigma=4.624)

Match 3 :
Ratings before match:
2. Simon - trueskill.Rating(mu=26.856, sigma=4.937)
3. Julian - trueskill.Rating(mu=24.544, sigma=4.878)
1. Sebastian - trueskill.R

/var/folders/cs/tjcwkq_55mb7_g2dhb5yh4980000gn/T/ipykernel_25379/3508643506.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  vis_df = pd.concat([vis_df, current_match, previous_matches], ignore_index=True)


In [7]:
# display(vis_df)

In [8]:
def generate_color_palette(n, alpha=1.0):
    return [
        f"hsla({h}, 70%, 50%, {alpha})"
        for h in range(0, 360, int(360 / n))
    ]

In [9]:
vis_df['Match'] = vis_df['Match'].astype(int)
vis_df = vis_df.sort_values(['Commander', 'Match'])

fig = go.Figure()

commanders = vis_df['Commander'].unique()
base_colors = generate_color_palette(len(commanders), alpha=1.0)
band_colors = generate_color_palette(len(commanders), alpha=0.1)

# Get distinct colors for each commander using Plotly's qualitative color palette
color_map_line = dict(zip(commanders, base_colors))
color_map_band = dict(zip(commanders, band_colors))

for commander in commanders:
    data = vis_df[vis_df['Commander'] == commander].copy()
    data = data.sort_values('Match')

    color_line = color_map_line[commander]
    color_band = color_map_band[commander]
    # color_rgba = color.replace('rgb', 'rgba').replace(')', ', 0.15)')  # Transparent fill

    upper = data['TrueSkill Rating'] + data['TrueSkill StDev']
    lower = data['TrueSkill Rating'] - data['TrueSkill StDev']

    # Line trace
    fig.add_trace(go.Scatter(
        x=data['Match'], y=data['TrueSkill Rating'],
        mode='lines',
        name=commander,
        legendgroup=commander,
        line=dict(width=2, color=color_line),
        hoverinfo='x+y+name'
    ))

    # Confidence band
    fig.add_trace(go.Scatter(
        x=pd.concat([data['Match'], data['Match'][::-1]]),
        y=pd.concat([upper, lower[::-1]]),
        fill='toself',
        fillcolor=color_band,
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo='skip',
        showlegend=False,
        legendgroup=commander
    ))

fig.update_layout(
    title='Commander TrueSkill Ratings with Confidence Bands',
    xaxis_title='Match',
    yaxis_title='TrueSkill Rating',
    template='plotly_white',
    legend_title='Commander',
    hovermode='x unified'
)

fig.show()